In [26]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
containerName = "tokyoolympiccontainer"
storageAccountName = "tokyoolympicdata"
accountkey = "please add access key from azure storage" #Copied from Accesskeys
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"
spark.conf.set("fs.azure.account.key.{storage}.dfs.core.windows.net".format(storage=storageAccountName), accountkey)
PATH_TEMPLATE = "abfss://{container}@{storage}.dfs.core.windows.net"
RAW_PATH = PATH_TEMPLATE.format(container=containerName, storage=storageAccountName)
RAW_FOLDER_PATH = 'rawdata'
INPUT_PATH=RAW_PATH+RAW_FOLDER_PATH
OUTPUT_PATH=RAW_PATH+'transformeddata'

In [ ]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [ ]:
athletes = spark.read.csv(INPUT_PATH+'/athletes',sep=',',inferSchema=True,header=True)athletes.csv")
coaches = spark.read.csv(INPUT_PATH+'/coaches',sep=',',inferSchema=True,header=True)
entriesgender = spark.read.csv(INPUT_PATH+'entriesgender',sep=',',inferSchema=True,header=True)
medals = spark.read.csv(INPUT_PATH+'/medals',sep=',',inferSchema=True,header=True)
teams = spark.read.csv(INPUT_PATH+'/teams',sep=',',inferSchema=True,header=True)

In [7]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [8]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [9]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [10]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [11]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [12]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [15]:
entriesgender = entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [16]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [17]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [18]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [19]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [20]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [27]:
# Clean up and standardize the data
athletes_transformed = athletes.withColumnRenamed("PersonName", "Name") \
                                  .withColumn("AthleteID", monotonically_increasing_id()) \
                                  .withColumn("Gender", when(col("Discipline").like("%Female%"), "Female").otherwise("Male")) \
                                  .withColumn("MedalCount", lit(0))  # Add logic to calculate the MedalCount if necessary

# Example transformation to count athletes by country and discipline
athletes_country_discipline = athletes_transformed.groupBy("Country", "Discipline").count()


In [29]:
athletes_country_discipline.show()

+--------------------+-------------------+-----+
|             Country|         Discipline|count|
+--------------------+-------------------+-----+
|Syrian Arab Republic|      Weightlifting|    1|
|             Belgium|               Golf|    3|
|               India|            Sailing|    4|
|             Ireland|          Badminton|    1|
|           Indonesia|             Rowing|    1|
|         Philippines|               Golf|    3|
|           Lithuania|Artistic Gymnastics|    1|
|               Niger|               Judo|    1|
|           Venezuela|             Rowing|    2|
| Republic of Moldova|          Athletics|    5|
|            Honduras|               Judo|    1|
|              Canada|          Wrestling|    4|
|             Senegal|          Wrestling|    1|
|             Comoros|          Athletics|    2|
|             Tunisia|            Archery|    2|
|         Puerto Rico|               Judo|    3|
|               Ghana|           Swimming|    2|
|             Algeri

In [48]:
# Clean up and add a unique ID for coaches
coaches=coaches.dropna()
coaches_transformed = coaches.withColumnRenamed("Name", "CoachName") \
                                .withColumn("CoachID", monotonically_increasing_id()) \
                                .withColumn("AthleteCount", lit(0))  # Logic to calculate athlete count if needed

# Group by discipline and event, and count number of coaches
coaches_discipline_event = coaches_transformed.groupBy("Discipline", "Event").count()


In [49]:
coaches_discipline_event.show()

+-----------------+--------+-----+
|       Discipline|   Event|count|
+-----------------+--------+-----+
|           Hockey|   Women|   18|
|       Basketball|   Women|   12|
|Artistic Swimming|    Duet|   27|
|       Volleyball|     Men|   12|
|       Volleyball|   Women|   12|
|         Handball|   Women|   12|
|         Football|   Women|   12|
|     Rugby Sevens|   Women|   12|
|Baseball/Softball|Softball|   20|
|         Football|     Men|   16|
|           Hockey|     Men|   18|
|Baseball/Softball|Baseball|    6|
|       Basketball|     Men|   12|
|         Handball|     Men|   12|
|       Water Polo|     Men|   12|
|Artistic Swimming|    Team|   14|
|       Water Polo|   Women|   10|
|     Rugby Sevens|     Men|   12|
+-----------------+--------+-----+



In [51]:
# Add a ratio column for gender distribution
entries_transformed = entriesgender.withColumn("FemaleMaleRatio", col("Female") / col("Male")) \
                                .withColumn("GenderBalance", when(col("Female") == col("Male"), "Balanced")
                                            .otherwise("Imbalanced"))

# Example transformation: Calculate the gender participation ratio for each discipline
entries_gender_distribution = entries_transformed.select("Discipline", "Female", "Male", "FemaleMaleRatio", "GenderBalance")


In [52]:
entries_gender_distribution.show()

+--------------------+------+----+------------------+-------------+
|          Discipline|Female|Male|   FemaleMaleRatio|GenderBalance|
+--------------------+------+----+------------------+-------------+
|      3x3 Basketball|    32|  32|               1.0|     Balanced|
|             Archery|    64|  64|               1.0|     Balanced|
| Artistic Gymnastics|    98|  98|               1.0|     Balanced|
|   Artistic Swimming|   105|   0|              NULL|   Imbalanced|
|           Athletics|   969|1072|0.9039179104477612|   Imbalanced|
|           Badminton|    86|  87|0.9885057471264368|   Imbalanced|
|   Baseball/Softball|    90| 144|             0.625|   Imbalanced|
|          Basketball|   144| 144|               1.0|     Balanced|
|    Beach Volleyball|    48|  48|               1.0|     Balanced|
|              Boxing|   102| 187|0.5454545454545454|   Imbalanced|
|        Canoe Slalom|    41|  41|               1.0|     Balanced|
|        Canoe Sprint|   123| 126|0.976190476190

In [54]:
# Calculate the percentage of gold, silver, and bronze medals
medals_transformed = medals.withColumn("GoldPercentage", col("Gold") / col("Total")) \
                               .withColumn("SilverPercentage", col("Silver") / col("Total")) \
                               .withColumn("BronzePercentage", col("Bronze") / col("Total"))

# Aggregate medals data by country
medals_by_country = medals_transformed.groupBy("TeamCountry").agg(
    sum("Gold").alias("TotalGold"),
    sum("Silver").alias("TotalSilver"),
    sum("Bronze").alias("TotalBronze"),
    sum("Total").alias("TotalMedals")
)

In [57]:
medals_by_country.show()

+--------------------+---------+-----------+-----------+-----------+
|         TeamCountry|TotalGold|TotalSilver|TotalBronze|TotalMedals|
+--------------------+---------+-----------+-----------+-----------+
|       Côte d'Ivoire|        0|          0|          1|          1|
|Islamic Republic ...|        3|          2|          2|          7|
|      Chinese Taipei|        2|          4|          6|         12|
|              Sweden|        3|          6|          0|          9|
|   Republic of Korea|        6|          4|         10|         20|
|         Philippines|        1|          2|          1|          4|
|            Malaysia|        0|          1|          1|          2|
|                Fiji|        1|          0|          1|          2|
|              Turkey|        2|          2|          9|         13|
|             Germany|       10|         11|         16|         37|
|              Jordan|        0|          1|          1|          2|
|              France|       10|  

In [58]:
# Add a unique ID for each team and calculate the total participants in the team
teams_transformed = teams.withColumn("TeamID", monotonically_increasing_id()) \
                            .withColumn("TotalParticipants", lit(0))  # Logic to count participants in the team if needed

# Aggregate teams by country and discipline
teams_country_discipline = teams_transformed.groupBy("Country", "Discipline").count()


In [59]:
teams_country_discipline.show()

+--------------------+-------------------+-----+
|             Country|         Discipline|count|
+--------------------+-------------------+-----+
|             Tunisia|            Archery|    1|
|          Kazakhstan|  Artistic Swimming|    1|
|              Serbia|         Water Polo|    1|
|             Belarus|            Archery|    1|
|      Czech Republic|          Athletics|    1|
|United States of ...|       Table Tennis|    2|
|         Netherlands|            Archery|    2|
|        South Africa|          Athletics|    2|
|              Latvia|   Beach Volleyball|    2|
|              Canada|         Basketball|    1|
|People's Republic...|           Football|    1|
|          Uzbekistan|Rhythmic Gymnastics|    1|
|           Australia|           Swimming|    7|
|              France|            Archery|    2|
|United States of ...|Artistic Gymnastics|    2|
|              Canada|      Cycling Track|    4|
|             Tunisia|            Fencing|    1|
|         Netherland

In [68]:
# Rename the columns during the join to avoid ambiguity
athletes_transformed = athletes.withColumnRenamed("Country", "AthleteCountry") \
                                .withColumnRenamed("PersonName", "AthleteName")
coaches_transformed = coaches.withColumnRenamed("Country", "CoachCountry") \
                             .withColumnRenamed("Name", "CoachName")


# Now perform the join, specifying the new column names
joined_df = athletes_transformed.join(coaches_transformed, 
                                      athletes_transformed.AthleteCountry == coaches_transformed.CoachCountry, 
                                      "inner")

# You can now refer to the columns without ambiguity
joined_df.select("AthleteName", "AthleteCountry", "CoachName", "CoachCountry").show()


+-----------------+--------------+--------------------+------------+
|      AthleteName|AthleteCountry|           CoachName|CoachCountry|
+-----------------+--------------+--------------------+------------+
|  AALERUD Katrine|        Norway|  HERGEIRSSON Thorir|      Norway|
|  AALERUD Katrine|        Norway|     BERGE Christian|      Norway|
|      ABAD Nestor|         Spain|VIVER ARZA Carlos...|       Spain|
|      ABAD Nestor|         Spain|      SOYEZ Frederic|       Spain|
|      ABAD Nestor|         Spain|     SCARIOLO Sergio|       Spain|
|      ABAD Nestor|         Spain| RIBERA ROMANS Jordi|       Spain|
|      ABAD Nestor|         Spain|REQUENA PERICAS J...|       Spain|
|      ABAD Nestor|         Spain|REQUENA PERICAS J...|       Spain|
|      ABAD Nestor|         Spain|OCA GAIA Miguel A...|       Spain|
|      ABAD Nestor|         Spain|MONDELO GARCIA Lucas|       Spain|
|      ABAD Nestor|         Spain| MARTIN LOZANO David|       Spain|
|      ABAD Nestor|         Spain|

In [69]:
athletes_transformed.write.mode("overwrite").parquet(f"{OUTPUT_PATH}/athletes_transformed")
coaches_transformed.write.mode("overwrite").parquet(f"{OUTPUT_PATH}/coaches_transformed")
entries_transformed.write.mode("overwrite").parquet(f"{OUTPUT_PATH}/entries_transformed")
medals_transformed.write.mode("overwrite").parquet(f"{OUTPUT_PATH}/medals_transformed")
teams_transformed.write.mode("overwrite").parquet(f"{OUTPUT_PATH}/teams_transformed")

249